## Assigment 3


In [1]:
require 'rest-client'

def fetch (url, headers = {accept: "*/*"}, user = "", pass="")
    response = RestClient::Request.execute({
      method: :get,
      url: url.to_s,
      user: user,
      password: pass,
      headers: headers})
    
    return response
  
    rescue RestClient::ExceptionWithResponse => e
    $stderr.puts e.inspect
    response = false
    return response
  
    rescue RestClient::Exception => e
    $stderr.puts e.inspect
    response = false
    return response
  
    rescue Exception => e
    $stderr.puts e.inspect
    response = false
    return response  
end


:fetch

In [ ]:
require 'bio'
response = fetch('http://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=ensemblgenomesgene&format=embl&id=At3g54340')
if response
  record = response.body
  sequence = Bio::EMBL.new(record).to_biosequence
  #$contains = 0
  sequence.features.each do |feature|
    if feature.feature == "exon"
      strand = feature.position.match(/complement/)
      exon_region = feature.position.match(/\(([^\(]+)\)/)[1]
      exon = Bio::Feature.new('exon',exon_region)
      exon.append(Bio::Feature::Qualifier.new('gene', 'At3g54340'))
      feature.qualifiers.each do |qualifier|
        exon_ID = qualifier.value.match(/(exon_id=((.*?)exon\d))/)
        puts exon_ID
        exon.append(Bio::Feature::Qualifier.new('exon_ID', exon_ID[2]))
        if not strand.nil?
          exon.append(Bio::Feature::Qualifier.new('strand', '-'))
          exon_region_limit1 = exon_region.split('..')[0]
          exon_region_limit2 = exon_region.split('..')[1]
          exon_sequence = sequence.subseq(exon_region_limit1.to_i, exon_region_limit2.to_i).complement
          exon.append(Bio::Feature::Qualifier.new('sequence', exon_sequence))
          exon.qualifiers.each do |x|
            if x.qualifier == "sequence" and not x.value.gsub(/(cttctt)+/).map{Regexp.last_match.begin(0)}.empty?
              match_positions = x.value.gsub(/(cttctt)+/).map{Regexp.last_match.begin(0)}
              match_begin = match_positions.map(&:succ)
              match_end = x.value.gsub(/(cttctt)+/).map{Regexp.last_match.end(0)}
              i = 0
              all_match = Array.new
              while i < match_begin.length
                match = Bio::Feature.new('match',match_begin[i]..match_end[i])
                match.append(Bio::Feature::Qualifier.new('sequid', 'some_gene'))
                match.append(Bio::Feature::Qualifier.new('source', 'BioRuby'))
                match.append(Bio::Feature::Qualifier.new('type', 'exon'))                
                match.append(Bio::Feature::Qualifier.new('score', '.'))
                match.append(Bio::Feature::Qualifier.new('phase', '.'))
                match.append(Bio::Feature::Qualifier.new('attribute1', 'repetitive_motif=CTTCTT'))
                match.append(Bio::Feature::Qualifier.new('attribute2', exon_ID[2]))
                all_match << match
                i = i +1
                #$contains += 1
              end
              #write_output(all_match)
            end 
          end
        else
          exon.append(Bio::Feature::Qualifier.new('strand', '+'))
          exon_region_limit1 = exon_region.split('..')[0]
          exon_region_limit2 = exon_region.split('..')[1]
          exon_sequence = sequence.subseq(exon_region_limit1.to_i, exon_region_limit2.to_i)
          exon.append(Bio::Feature::Qualifier.new('sequence', exon_sequence))
          exon.qualifiers.each do |x|
            if x.qualifier == "sequence" and not x.value.gsub(/(cttctt)+/).map{Regexp.last_match.begin(0)}.empty?
              match_positions = x.value.gsub(/(cttctt)+/).map{Regexp.last_match.begin(0)}
              match_begin = match_positions.map(&:succ)
              match_end = x.value.gsub(/(cttctt)+/).map{Regexp.last_match.end(0)}
              i = 0
              all_match = Array.new
              while i < match_begin.length
                match = Bio::Feature.new('match',match_begin[i]..match_end[i])
                match.append(Bio::Feature::Qualifier.new('sequid', 'some_gene'))
                match.append(Bio::Feature::Qualifier.new('source', 'BioRuby'))
                match.append(Bio::Feature::Qualifier.new('type', 'exon'))               
                match.append(Bio::Feature::Qualifier.new('score', '.'))
                match.append(Bio::Feature::Qualifier.new('phase', '.'))
                match.append(Bio::Feature::Qualifier.new('attribute1', 'repetitive_motif=CTTCTT'))
                match.append(Bio::Feature::Qualifier.new('attribute2', exon_ID[2]))
                all_match << match
                i = i +1 
                #$contains += 1
              end
              #write_output1(all_match)
            end 
          end
        end
      end   
    end
  end
  #if $contains == 0
  #write_no_repetition(some_gene)
end

In [ ]:
a = "heeelloheelloohello".gsub(/e(e)+/).map{Regexp.last_match.end(0)}
#b = a.map(&:succ)
a.class
for i in a 
  puts i
end
a.length

In [ ]:
    case 
      when feature.feature == "gene"
      feature.qualifiers.each do |x|
        if x.qualifier == "note"
          source = x.value.match(/Source:(.*?);/)[1]
          if not source[1].nil?
            source = Bio::Feature::Qualifier.new('source', source)
          else
            source = Bio::Feature::Qualifier.new('source', 'unknown')
          end
        end
      end

In [152]:
require 'bio'
def search_exons(some_gene)
    response = fetch("http://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=ensemblgenomesgene&format=embl&id=#{some_gene}")
    if response
      record = response.body
      sequence = Bio::EMBL.new(record).to_biosequence
      $contains = 0
      sequence.features.each do |feature|
        if feature.feature == "exon"
          exon_region = feature.position.match(/\d+\.\.\d+/).to_s
          puts feature.position
          exon = Bio::Feature.new('exon',exon_region)
          exon.append(Bio::Feature::Qualifier.new('gene', some_gene))
          feature.qualifiers.each do |qualifier|
            exon_ID = qualifier.value.match(/(exon_id=((.*?)exon\d))/)[2].to_s
            agi = exon_ID.split('.')[0]
            if agi.to_s == some_gene.upcase
              exon.append(Bio::Feature::Qualifier.new('number', exon_ID))
              if not feature.position.match(/complement/)
                exon.append(Bio::Feature::Qualifier.new('strand', '+'))
                exon_region_limit1 = exon_region.split('..')[0]
                exon_region_limit2 = exon_region.split('..')[1]
                exon_sequence = sequence.subseq(exon_region_limit1.to_i, exon_region_limit2.to_i)
                exon.append(Bio::Feature::Qualifier.new('sequence', exon_sequence))
                exon.qualifiers.each do |x|
                  puts x.qualifier
                  if x.qualifier == "sequence" and not x.value.gsub(/ctt(ctt)+/).map{Regexp.last_match.begin(0)}.empty?
                    match_positions = x.value.gsub(/ctt(ctt)+/).map{Regexp.last_match.begin(0)}
                    match_begin = match_positions.map(&:succ)
                    match_end = x.value.gsub(/ctt(ctt)+/).map{Regexp.last_match.end(0)}
                    i = 0
                    all_match = Array.new
                    while i < match_begin.length
                      match = Bio::Feature.new('match',match_begin[i]..match_end[i])
                      match.append(Bio::Feature::Qualifier.new('sequid', some_gene))
                      match.append(Bio::Feature::Qualifier.new('strand', '+'))
                      match.append(Bio::Feature::Qualifier.new('attribute2', exon_ID))
                      all_match << match
                      i = i +1
                      $contains += 1
                      puts $contains
                    end
                    ##write_gff1(all_match)
                  end
                end
              else
                exon.append(Bio::Feature::Qualifier.new('strand', '-'))
                exon_region_limit1 = exon_region.split('..')[0]
                exon_region_limit2 = exon_region.split('..')[1]
                puts sequence.subseq(exon_region_limit1.to_i, exon_region_limit2.to_i)
                exon_sequence = sequence.subseq(exon_region_limit1.to_i, exon_region_limit2.to_i).complement
                exon.append(Bio::Feature::Qualifier.new('sequence', exon_sequence))
                exon.qualifiers.each do |x|
                  if x.qualifier == "sequence" and not x.value.gsub(/(cttctt)+/).map{Regexp.last_match.begin(0)}.empty?
                    match_positions = x.value.gsub(/(cttctt)+/).map{Regexp.last_match.begin(0)}
                    match_begin = match_positions.map(&:succ)
                    match_end = x.value.gsub(/(cttctt)+/).map{Regexp.last_match.end(0)}
                    i = 0
                    all_match = Array.new
                    while i < match_begin.length
                      match = Bio::Feature.new('match',match_begin[i]..match_end[i])
                      match.append(Bio::Feature::Qualifier.new('sequid', some_gene))
                      match.append(Bio::Feature::Qualifier.new('strand', '-'))
                      match.append(Bio::Feature::Qualifier.new('attribute2', exon_ID))
                      all_match << match
                      i = i +1
                      $contains += 1
                      puts $contains
                      end
                    ##write_gff1(all_match)
                  end
                end
              end
            else
              next
            end
            end
          end
        end
      end
      if $contains == 0
        write_no_cttctt(some_gene)
        puts $contains
        puts "soy bobo"
      end
    end


:search_exons

In [153]:
search_exons('AT1g07010')

F10K1_1:83276..83551
F10K1_1:83674..84216
2085..2441
gene
number
strand
sequence
1922..1991
gene
number
strand
sequence
953..1165
gene
number
strand
sequence
1..335
gene
number
strand
sequence
1
399..511
gene
number
strand
sequence
1750..1838
gene
number
strand
sequence
1255..1390
gene
number
strand
sequence
2085..2472
gene
number
strand
sequence
707..808
gene
number
strand
sequence
83..335
gene
number
strand
sequence
2
695..808
gene
number
strand
sequence
594..626
gene
number
strand
sequence
1477..1652
gene
number
strand
sequence
55..335
gene
number
strand
sequence
3
426..526
gene
number
strand
sequence
complement(F10K1_1:83807..84216)


In [21]:
a = "83674..84216"
a.split("..")[1]

"84216"